In [1]:
import pandas as pd

In [2]:
def move_first_row_to_header(population_dataframe):
    df = population_dataframe.copy()
    new_header = df.iloc[0]
    df = df[1:]
    new_header[0] = "0"
    new_header.astype(int)
    new_header = [int(ele) for ele in new_header]
    new_header[0] = "County"
    df.columns = new_header
    return df

In [3]:
# This function only applies for data format of year 2000-2009
def structure_modification_for_old_xls(population_dataframe, state_name):

    df = population_dataframe.copy()
    # Drop unecessary columns and rows
    df.drop(df.columns[[1, 12, 13]], axis=1, inplace=True)
    df.drop(df.index[[0, 1, 3]], inplace=True)

    # Make row0 the header with a little modification tricks
    df = move_first_row_to_header(df)

    # Reset Index
    df.reset_index(inplace=True)
    df.drop("index", axis=1, inplace=True)

    # Some finetune
    df["County"] = df["County"].str[1:]
    df["State"] = state_name
    df = df.drop(df[df[2000].isnull()].index)

    return df

In [4]:
# This function only applies for data format of year 2010-2019
def structure_modification_for_new_xlsx(population_dataframe, state_name):

    df = population_dataframe.copy()

    # Drop unecessary columns and rows
    df.drop(df.columns[[1, 2]], axis=1, inplace=True)
    df.drop(df.index[[0, 1, 3]], inplace=True)

    # Make row0 the header with a little modification tricks
    df = move_first_row_to_header(df)

    # Reset Index
    df.reset_index(inplace=True)
    df.drop("index", axis=1, inplace=True)

    # Some finetune
    df["County"] = df["County"].str.split(", ").str[0]
    df["County"] = df["County"].str[1:]
    df["State"] = state_name
    df = df.drop(df[df[2010].isnull()].index)

    return df

In [5]:
# Read population data for all target states
state_list = ["FL", "TX", "WA", "NC", "SC", "PA"]

old_xls_list = []
new_xlsx_list = []
for state in state_list:
    # Read data from 2000 to 2009
    old_state_population = pd.read_excel(f"../00_source_data/Population/{state}_population_2000_2009.xls")
    old_state_population_modified = structure_modification_for_old_xls(old_state_population, state)
    old_xls_list.append(old_state_population_modified)
    # Read data from 2010 to 2019
    new_state_population = pd.read_excel(f"../00_source_data/Population/{state}_population_2010_2019.xlsx")
    new_state_population_modified = structure_modification_for_new_xlsx(new_state_population, state)
    # new_state_population_modified["County"] = new_state_population_modified["County"].str[1:]
    new_xlsx_list.append(new_state_population_modified)


popuplation_2000 = pd.concat(old_xls_list)
popuplation_2010 = pd.concat(new_xlsx_list)
popuplation_2000.sample(10)


,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,State
13,Centre County,135965,137620.0,140821.0,142926.0,144238.0,145543.0,148734.0,149498.0,151645.0,153052.0,PA
153,McCulloch County,8175,8034.0,7927.0,8033.0,8164.0,8117.0,8183.0,8105.0,8203.0,8365.0,TX
80,Rutherford County,63064,63671.0,63924.0,64475.0,64692.0,65126.0,65855.0,66305.0,67145.0,67561.0,NC
12,Cabarrus County,132231,136463.0,140054.0,143336.0,146429.0,150797.0,157735.0,165717.0,171915.0,175993.0,NC
87,Goliad County,7007,7020.0,7051.0,7135.0,7071.0,7070.0,7121.0,7142.0,7164.0,7157.0,TX
231,Uvalde County,25900,25939.0,26309.0,26258.0,26205.0,26368.0,26301.0,26217.0,26206.0,26223.0,TX
53,Schuylkill County,149994,149175.0,148705.0,147792.0,147081.0,146976.0,148005.0,148387.0,148463.0,148358.0,PA
30,Lee County,20088,19957.0,19970.0,20005.0,20157.0,20079.0,20001.0,19836.0,19697.0,19352.0,SC
159,Mason County,3734,3743.0,3725.0,3762.0,3811.0,3813.0,3862.0,3897.0,3876.0,3982.0,TX
83,Stanly County,58246,58674.0,58689.0,58748.0,58638.0,58931.0,59185.0,59849.0,60418.0,60496.0,NC


In [6]:
popuplation_2010.sample(10)

,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
40,Guilford County,489618,494953.0,500514.0,506400.0,512233.0,517053.0,524265.0,528243.0,532607.0,537174.0,NC
38,Clay County,10737,10668.0,10523.0,10455.0,10377.0,10359.0,10251.0,10484.0,10448.0,10471.0,TX
182,Panola County,23772,24007.0,24001.0,23790.0,23750.0,23667.0,23419.0,23211.0,23142.0,23194.0,TX
35,Chambers County,35456,35699.0,36508.0,37367.0,38296.0,39028.0,40163.0,41269.0,42227.0,43837.0,TX
37,Childress County,7070,7027.0,7100.0,7055.0,7196.0,7169.0,7204.0,7269.0,7315.0,7306.0,TX
167,Mitchell County,9412,9396.0,9329.0,8999.0,9070.0,8856.0,8474.0,8215.0,8525.0,8545.0,TX
32,Lafayette County,8808,8798.0,8791.0,8819.0,8862.0,8724.0,8747.0,8602.0,8691.0,8422.0,FL
16,Dillon County,32078,31750.0,31530.0,31411.0,31296.0,31141.0,30719.0,30496.0,30612.0,30479.0,SC
13,Dixie County,16400,16406.0,16153.0,16084.0,16041.0,16353.0,16465.0,16615.0,16685.0,16826.0,FL
94,Watauga County,50972,51626.0,52062.0,52295.0,52395.0,53093.0,54140.0,55181.0,56034.0,56177.0,NC


In [179]:
# Data validity check
def validity_check(df):
    for column in df.columns:
        assert not df[column].isnull().any()

In [180]:
validity_check(popuplation_2000)
validity_check(popuplation_2010)

In [181]:
merged_population = popuplation_2000.merge(popuplation_2010, validate="1:1", indicator=True)
assert merged_population[merged_population["_merge"] != "both"].empty
merged_population.columns = merged_population.columns.astype(str)
merged_population.head()

,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,_merge
0,Alachua County,218611,221718.0,224614.0,227022.0,229867.0,233756.0,239506.0,242685.0,244888.0,...,249834.0,251520.0,252475.0,255456.0,259052.0,263959.0,266309.0,268851.0,269043.0,both
1,Baker County,22374,22620.0,23298.0,23555.0,24142.0,24832.0,25571.0,26212.0,26725.0,...,27049.0,27053.0,27009.0,27122.0,27355.0,27884.0,28254.0,28353.0,29210.0,both
2,Bay County,148393,150207.0,152741.0,155044.0,158804.0,162917.0,165644.0,165345.0,166267.0,...,169555.0,171757.0,174596.0,178289.0,181488.0,183634.0,184736.0,186240.0,174705.0,both
3,Bradford County,26064,26083.0,26306.0,27035.0,27703.0,28098.0,28506.0,28825.0,28961.0,...,28431.0,27051.0,26802.0,26552.0,26748.0,26740.0,27142.0,27752.0,28201.0,both
4,Brevard County,477819,486429.0,495425.0,504847.0,518722.0,529907.0,535138.0,539719.0,542378.0,...,544359.0,546966.0,550255.0,555548.0,565746.0,576874.0,587769.0,595203.0,601942.0,both


In [182]:
merged_population.to_parquet("../20_intermediate_files/Population_2000-2019.gzip", compression="gzip")
merged_population.to_csv("../20_intermediate_files/Population_2000-2019.csv")